In [1]:
import os
import sys
import open3d as o3d

sys.path.append("C:/Users/lucas/Desktop/UPC/MIT/pyfbx_i42-master/")

In [2]:
def fbx_to_py(fn):
    import pyfbx.parse_bin
    fbx_root_elem, fbx_version = pyfbx.parse_bin.parse(fn, use_namedtuple=True)
    py_data = fbx_root_elem, fbx_version
    return py_data


def py_to_fbx(fn, py_data):
    import pyfbx.encode_bin
    from pyfbx.encode_bin import FBXElem
    from pyfbx import data_types

    fbx_root_elem_in, fbx_version_in = py_data

    # re-build data
    def elem_conv_recursive(ele_in, ele_out):
        for i, data in enumerate(ele_in.props):
            data_type = ele_in.props_type[i]

            if data_type == data_types.BOOL:
                ele_out.add_bool(data)
            elif data_type == data_types.INT16:
                ele_out.add_int16(data)
            elif data_type == data_types.INT32:
                ele_outt.add_int32(data)
            elif data_type == data_types.INT64:
                ele_out.add_int64(data)
            elif data_type == data_types.FLOAT32:
                ele_out.add_float32(data)
            elif data_type == data_types.FLOAT64:
                ele_out.add_float64(data)
            elif data_type == data_types.BYTES:
                ele_out.add_bytes(data)
            elif data_type == data_types.STRING:
                ele_out.add_string(data)
            elif data_type == data_types.INT32_ARRAY:
                ele_out.add_int32_array(data)
            elif data_type == data_types.INT64_ARRAY:
                ele_out.add_int64_array(data)
            elif data_type == data_types.FLOAT32_ARRAY:
                ele_out.add_float32_array(data)
            elif data_type == data_types.FLOAT64_ARRAY:
                ele_out.add_float64_array(data)
            elif data_type == data_types.BOOL_ARRAY:
                ele_out.add_bool_array(data)
            elif data_type == data_types.BYTE_ARRAY:
                ele_out.add_byte_array(data)
            else:
                raise Exception("Unknown type %r" % data_type)

        for ele_iter_in in ele_in.elems:
            ele_iter_out = FBXElem(ele_iter_in.id)
            elem_conv_recursive(ele_iter_in, ele_iter_out)
            ele_out.elems.append(ele_iter_out)

    fbx_root_elem_out = FBXElem(b'')
    elem_conv_recursive(fbx_root_elem_in, fbx_root_elem_out)

    pyfbx.encode_bin.write(fn, fbx_root_elem_out, fbx_version_in)

In [3]:
fn_in = "C:/Users/lucas/Downloads/Borrar/fbx.fbx"
fn_in = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/intrinsics_extrinsics/data/20210719_2/recording_2021-07-19_17-01-51.fbx"
#fn_in = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/intrinsics_extrinsics/data/20210722/recording_2021-07-22_13-25-10.fbx"

py_data = fbx_to_py(fn_in)
a, num = py_data
objects = a.elems[8].elems

In [4]:
a.elems[2]

FBXElem(id=b'CreationTime', props=[b'2021-07-19 17:01:51:482'], props_type=bytearray(b'S'), elems=[])

In [5]:
for i in range(11):
    print(i, a.elems[i].id)

0 b'FBXHeaderExtension'
1 b'FileId'
2 b'CreationTime'
3 b'Creator'
4 b'GlobalSettings'
5 b'Documents'
6 b'References'
7 b'Definitions'
8 b'Objects'
9 b'Connections'
10 b'Takes'


In [6]:
i = 28
name = objects[i].props[1].decode("utf-8").split('\x00')[0]
translation = objects[i].elems[1].elems[2].props[-3:]
rotation = objects[i].elems[1].elems[3].props[-3:]

rotation 

[81.67082977294922, 14.704727172851562, -15.851095199584961]

In [7]:
def euler_to_rotMat(yaw, pitch, roll):
    Rz_yaw = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw),  np.cos(yaw), 0],
        [          0,            0, 1]])
    Ry_pitch = np.array([
        [ np.cos(pitch), 0, np.sin(pitch)],
        [             0, 1,             0],
        [-np.sin(pitch), 0, np.cos(pitch)]])
    Rx_roll = np.array([
        [1,            0,             0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll),  np.cos(roll)]])
    # R = RzRyRx
    rotMat = np.dot(Rz_yaw, np.dot(Ry_pitch, Rx_roll))
    return rotMat

In [8]:
import numpy as np
import math as m
  
def Rx(theta):
    return np.matrix([[ 1, 0           , 0           ],
                   [ 0, m.cos(theta),-m.sin(theta)],
                   [ 0, m.sin(theta), m.cos(theta)]])
  
def Ry(theta):
    return np.matrix([[ m.cos(theta), 0, m.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-m.sin(theta), 0, m.cos(theta)]])
  
def Rz(theta):
    return np.matrix([[ m.cos(theta), -m.sin(theta), 0 ],
                   [ m.sin(theta), m.cos(theta) , 0 ],
                   [ 0           , 0            , 1 ]])



def get_rotation(rot, vec):
    psi, theta, phi = rot
    #phi, theta, psi = rot

    R = Rz(psi) * Ry(theta) * Rx(phi)
    #R = euler_to_rotMat()
    rot_vec = R @ vec
    
    return np.squeeze(np.asarray(rot_vec))

def get_finger(index):
    
    finger_01 = np.array(objects[index].elems[1].elems[2].props[-3:])
    finger_02 = np.array(objects[index + 1].elems[1].elems[2].props[-3:]) + finger_01
    finger_03 = np.array(objects[index + 2].elems[1].elems[2].props[-3:]) + finger_02
    finger_04 = np.array(objects[index + 3].elems[1].elems[2].props[-3:]) + finger_03
    
    rotation_01 = np.radians(objects[index].elems[1].elems[3].props[-3:])
    rotation_02 = np.radians(objects[index + 1].elems[1].elems[3].props[-3:]) 
    rotation_03 = np.radians(objects[index + 2].elems[1].elems[3].props[-3:]) 
    rotation_04 = np.radians(objects[index + 3].elems[1].elems[3].props[-3:]) 

    #finger_01 = get_rotation(rotation_01, finger_01)
    #finger_02 = get_rotation(rotation_02, finger_02) + finger_01
    #finger_03 = get_rotation(rotation_03, finger_03) + finger_02
    #finger_04 = get_rotation(rotation_04, finger_04) + finger_03

    finger = [finger_01, finger_02, finger_03, finger_04]
    
    return finger

In [9]:

all_fingers = get_finger(28) + get_finger(32) + get_finger(36) + get_finger(40) + get_finger(44)

hand_xyz = np.vstack((all_fingers))

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(hand_xyz)

mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()
mesh.scale(0.05, center=(0,0,0))

o3d.visualization.draw_geometries([pcd])

In [10]:
thumb_01_l = np.array(objects[28].elems[1].elems[2].props[-3:])
thumb_02_l = np.array(objects[29].elems[1].elems[2].props[-3:]) 
rotation = objects[29].elems[1].elems[3].props[-3:]

thumb_02_l = get_rotation(rotation, thumb_02_l - thumb_01_l) + thumb_01_l
thumb_03_l = np.array(objects[30].elems[1].elems[2].props[-3:]) + thumb_02_l
thumb_tip_l = np.array(objects[31].elems[1].elems[2].props[-3:]) + thumb_03_l

index_01_l = np.array(objects[32].elems[1].elems[2].props[-3:])
index_02_l = np.array(objects[33].elems[1].elems[2].props[-3:]) + index_01_l
index_03_l = np.array(objects[34].elems[1].elems[2].props[-3:]) + index_02_l
index_tip_l = np.array(objects[35].elems[1].elems[2].props[-3:]) + index_03_l

middle_01_l = np.array(objects[36].elems[1].elems[2].props[-3:])
middle_02_l = np.array(objects[37].elems[1].elems[2].props[-3:]) + middle_01_l
middle_03_l = np.array(objects[38].elems[1].elems[2].props[-3:]) + middle_02_l
middle_tip_l = np.array(objects[39].elems[1].elems[2].props[-3:]) + middle_03_l

ring_01_l = np.array(objects[40].elems[1].elems[2].props[-3:])
ring_02_l = np.array(objects[41].elems[1].elems[2].props[-3:]) + ring_01_l
ring_03_l = np.array(objects[42].elems[1].elems[2].props[-3:]) + ring_02_l
ring_tip_l = np.array(objects[43].elems[1].elems[2].props[-3:]) + ring_03_l

pinky_01_l = np.array(objects[44].elems[1].elems[2].props[-3:])
pinky_02_l = np.array(objects[45].elems[1].elems[2].props[-3:]) + pinky_01_l
pinky_03_l = np.array(objects[46].elems[1].elems[2].props[-3:]) + pinky_02_l
pinky_tip_l = np.array(objects[47].elems[1].elems[2].props[-3:]) + pinky_03_l

all_fingers = [thumb_01_l, thumb_02_l, thumb_03_l, thumb_tip_l, 
               index_01_l, index_02_l, index_03_l, index_tip_l, 
               middle_01_l, middle_02_l, middle_03_l, middle_tip_l, 
               ring_01_l, ring_02_l, ring_03_l, ring_tip_l, 
               pinky_01_l, pinky_02_l, pinky_03_l, pinky_tip_l]


hand_xyz = np.vstack((all_fingers))